In [5]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf
import csv # Import the csv module

# Adjusted threshold for the new model. This threshold determines when an entire clip is considered anomalous.
# This threshold might need re-tuning for the frame-by-frame model.
ANOMALY_THRESHOLD = 1.49 # A much smaller threshold is expected for single-frame MSE

# Ensure output folders exist
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"): # Folder for reconstructed image outputs (JPEGs)
    os.makedirs("reconstructed_frames")
if not os.path.exists("output_predictions_csv"): # New folder for raw model predictions saved as CSV
    os.makedirs("output_predictions_csv")

frame_count_processed = 0 # Renamed from output_num to reflect frame-by-frame processing
anomaly_count = 0

# Define the main CSV log file path and header
csv_file_path = "anomaly_detection_log_frame_by_frame.csv" # New CSV name for clarity
csv_header = [
    "frame_number", # Changed from clip_number
    "overall_loss",
    "status",
    "pixel_diff_mean",
    "pixel_diff_max",
    "pixel_diff_std",
    "reconstructed_mean_pixel_value",
    "reconstructed_std_pixel_value",
    "reconstructed_min_pixel_value",
    "reconstructed_max_pixel_value"
]
csv_data = [] # List to store data rows before writing to main CSV

def mean_squared_loss(x1, x2):
    """
    Calculates the mean squared loss between two arrays.
    Returns the mean distance.
    This function is now used for single frames.
    """
    # Ensure inputs are flattened for consistent mean distance calculation
    # x1 and x2 will be (1, H, W, 1) -> flatten to (H*W)
    difference = x1.flatten() - x2.flatten()
    sq_difference = difference ** 2
    distance = np.sqrt(sq_difference.sum())
    n_samples = np.prod(x1.shape[1:]) # H*W*C
    mean_distance = distance / n_samples
    return mean_distance

def combined_loss(y_true, y_pred):
    """
    Custom combined loss function for the Keras model.
    Assumes mean squared error is the primary component.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocess frame for the new model architecture.
    The new model expects better normalized inputs.
    """
    # Resize frame to the model's expected input dimensions
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    # Convert to grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to [0, 1]
    gray = gray.astype(np.float32) / 255.0
    # Further normalize to [-1, 1] as per model's expectation
    gray = (gray - 0.5) * 2
    return gray

# Load the trained Keras model (updated path for frame-by-frame model)
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model("../models/model_frame_by_frame.h5", custom_objects=custom_objects)
    print("Frame-by-frame model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model_frame_by_frame.h5' is in the '../models/' directory.")
    exit() # Exit if model cannot be loaded

# Open the video file for processing
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit() # Exit if video cannot be opened

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    ret, frame = cap.read() # Read one frame at a time
    if not ret:
        print("End of video or issue reading frame.")
        break

    frame_count_processed += 1 # Increment frame counter

    # Store the original color frame for display and saving
    original_frame_color = imutils.resize(frame, width=700, height=600)

    # Preprocess the single frame for model input
    processed_frame = preprocess_frame_for_new_model(frame)
    
    # Add batch and channel dimensions: (1, 232, 232, 1)
    model_input_frame = np.expand_dims(processed_frame, axis=0) # Add batch dim
    model_input_frame = np.expand_dims(model_input_frame, axis=-1) # Add channel dim

    # Predict with the model (output will be 1 reconstructed frame)
    output = model.predict(model_input_frame) # Output shape: (1, 232, 232, 1)

    # --- Save Model Prediction (Individual Reconstructed Frame as Image) ---
    reconstructed_frame_2d = output[0, :, :, 0] # Get the 2D reconstructed frame

    # Normalize to 0-255 and convert to 8-bit unsigned integer for saving as image
    reconstructed_frame_display = cv2.normalize(
        reconstructed_frame_2d, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U
    )
    # Resize for better viewing if desired, or keep original model output size
    reconstructed_frame_display = cv2.resize(
        reconstructed_frame_display, (original_frame_color.shape[1], original_frame_color.shape[0])
    )
    cv2.imwrite(f"reconstructed_frames/frame_{frame_count_processed:05d}_reconstructed.jpg", reconstructed_frame_display)
    # --- End Save Model Prediction (Image) ---

    # --- Save Raw Model Prediction as CSV (232x232 per frame) ---
    # Get the 2D frame (232, 232)
    frame_to_save_csv = output[0, :, :, 0]

    # Define the CSV file path for this specific frame
    per_frame_csv_path = f"output_predictions_csv/frame_{frame_count_processed:05d}_prediction.csv"

    with open(per_frame_csv_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write each row of the 232x232 frame to the CSV, formatted to 5 decimal places
        for row in frame_to_save_csv:
            formatted_row = [f"{pixel:.5f}" for pixel in row] # Format each pixel value
            csv_writer.writerow(formatted_row)

    # --- End Save Raw Model Prediction as CSV ---


    # Calculate the overall reconstruction loss for the current frame
    loss = mean_squared_loss(model_input_frame, output)
    # The scaling factor for loss might need adjustment for single-frame MSE
    # A smaller scaling factor or direct loss might be more appropriate.
    # For now, let's keep it scaled by 1000 for consistency with previous display,
    # but be aware that the numerical values will be much smaller.
    loss_scaled = loss * 1000
    loss_display = f"{loss_scaled:.5f}" # Formatted to 5 decimal points
    print(f"Frame {frame_count_processed} Loss: {loss_display}")

    # Create a copy of the original color frame to draw on
    display_frame = original_frame_color.copy()

    current_status = "Normal" # Default status
    pixel_diff_mean_val = 0.0
    pixel_diff_max_val = 0.0
    pixel_diff_std_val = 0.0

    # Initialize reconstructed output statistics for the main CSV log
    reconstructed_mean_pixel_value = np.mean(output)
    reconstructed_std_pixel_value = np.std(output)
    reconstructed_min_pixel_value = np.min(output)
    reconstructed_max_pixel_value = np.max(output)


    # Check if the overall frame loss indicates an anomaly
    # The ANOMALY_THRESHOLD will likely need significant adjustment for frame-by-frame MSE
    if loss_scaled > ANOMALY_THRESHOLD: # You will need to re-tune this threshold!
        current_status = "Abnormal"
        print(f"Status: Abnormal (Score = {loss_display}) - Detecting anomalous regions...")

        # --- Anomaly Region Detection and Visualization ---
        # Extract the 2D input and reconstructed frames
        input_frame_2d = model_input_frame[0, :, :, 0]
        reconstructed_frame_2d_for_diff = output[0, :, :, 0]

        # Calculate the absolute difference (reconstruction error) pixel by pixel
        diff_frame = np.abs(input_frame_2d - reconstructed_frame_2d_for_diff)

        # Calculate statistics for CSV
        pixel_diff_mean_val = np.mean(diff_frame)
        pixel_diff_max_val = np.max(diff_frame)
        pixel_diff_std_val = np.std(diff_frame)

        # Normalize diff_frame to [0, 1] for easier thresholding
        if diff_frame.max() - diff_frame.min() == 0: # Handle flat diff_frame
            diff_frame_normalized_01 = np.zeros_like(diff_frame, dtype=np.float32)
        else:
            diff_frame_normalized_01 = cv2.normalize(diff_frame, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)

        # Apply a threshold to create a binary mask of high error regions.
        # This threshold (0.25) will likely need re-tuning for single-frame analysis.
        _, anomaly_mask = cv2.threshold(diff_frame_normalized_01, 0.25, 255, cv2.THRESH_BINARY)
        anomaly_mask = anomaly_mask.astype(np.uint8) # Convert to 8-bit for colormap

        # Apply a colormap to the anomaly mask.
        heatmap = cv2.applyColorMap(anomaly_mask, cv2.COLORMAP_HOT)

        # Optional: Apply a slight blur to the heatmap for smoother regions
        heatmap = cv2.GaussianBlur(heatmap, (5, 5), 0)

        # Resize the heatmap to match the dimensions of the original display frame
        heatmap_resized = cv2.resize(heatmap, (display_frame.shape[1], display_frame.shape[0]))

        # Overlay the heatmap on the original color frame.
        display_frame = cv2.addWeighted(display_frame, 0.7, heatmap_resized, 0.3, 0)
        # --- End Anomaly Region Detection and Visualization ---

        # Add text overlay for status and save the frame
        cv2.putText(display_frame, f"Abnormal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) # Red color for abnormal
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}_abnormal.jpg", display_frame)
        anomaly_count += 1
    else:
        # If the frame is normal, just add text overlay and save the frame
        print(f"Status: Normal (Score = {loss_display})")
        cv2.putText(display_frame, f"Normal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) # Green color for normal
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}_normal.jpg", display_frame)

    # Append data for the current frame to the main CSV data list, formatted to 5 decimal points
    csv_data.append([
        frame_count_processed,
        f"{float(loss_display):.5f}", # Convert to float then format for consistency
        current_status,
        f"{pixel_diff_mean_val:.5f}",
        f"{pixel_diff_max_val:.5f}",
        f"{pixel_diff_std_val:.5f}",
        f"{reconstructed_mean_pixel_value:.5f}",
        f"{reconstructed_std_pixel_value:.5f}",
        f"{reconstructed_min_pixel_value:.5f}",
        f"{reconstructed_max_pixel_value:.5f}"
    ])

    # Display the processed frame in a window
    cv2.imshow("Video Playback with Anomaly Detection", display_frame)
    cv2.waitKey(1) # Wait for 1 millisecond (allows display update)

    # Print progress every 100 frames
    if frame_count_processed % 100 == 0:
        print(f"Processed {frame_count_processed} frames, found {anomaly_count} anomalies so far.")

    # Allow exiting the loop by pressing 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release video capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# --- Write main anomaly log data to CSV file after processing all frames ---
print(f"\nWriting overall anomaly log to {csv_file_path}...")
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header) # Write header
    csv_writer.writerows(csv_data) # Write all collected data rows
print("Overall anomaly log CSV file saved successfully.")
# --- End Write to CSV ---

print(f"\nProcessing complete!")
print(f"Total frames processed: {frame_count_processed}")
print(f"Total anomalies detected: {anomaly_count}")
if frame_count_processed > 0:
    anomaly_rate = (anomaly_count / frame_count_processed * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No frames were processed, cannot calculate anomaly rate.")


Frame-by-frame model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 0s 109ms/step
Frame 1 Loss: 1.50866
Status: Abnormal (Score = 1.50866) - Detecting anomalous regions...
1/1 [==============================] - 0s 19ms/step
Frame 2 Loss: 1.50698
Status: Abnormal (Score = 1.50698) - Detecting anomalous regions...
1/1 [==============================] - 0s 19ms/step
Frame 3 Loss: 1.50661
Status: Abnormal (Score = 1.50661) - Detecting anomalous regions...
1/1 [==============================] - 0s 17ms/step
Frame 4 Loss: 1.50590
Status: Abnormal (Score = 1.50590) - Detecting anomalous regions...
1/1 [==============================] - 0s 19ms/step
Frame 5 Loss: 1.50511
Status: Abnormal (Score = 1.50511) - Detecting anomalous regions...
1/1 [==============================] - 0s 18ms/step
Frame 6 Loss: 1.50423
Status: Abnormal (Score = 1.50423) - Detecting anomalous regions...
1/1 [==============================] - 0s 19ms/step
Frame 7 Loss: 1.50447
Stat